## We now attempt to find out if the way a SEC filing is done has an effect on the returns

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import re

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import scipy.stats as stats

# Remove warnings to keep outputs clean
warnings.filterwarnings("ignore")

PROCESSED_DATA_FOLDER = "../../data_untracked/processed"
ABNORMAL_CSV = "df_result_with_anomaly_scores.csv"

# COMPILED_DATA_FOLDER = "../../data_untracked/raw/sec_submissions/compiled"
# FOOTNOTES_CSV = "FOOTNOTES.csv"

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')

## Load required datasets
1. `df_result_with_anomaly_scores_sig.csv`: Transactions with their Cumulative Abnormal Returns (CAR)
2. `FOOTNOTES.csv`: Each transactions with their individual filing comments

In [ ]:
abnormal_transactions = pd.read_csv(f'{PROCESSED_DATA_FOLDER}/{ABNORMAL_CSV}')[["ACCESSION_NUMBER", "TRANS_CODE", "TRANS_ACQUIRED_DISP_CD", "anomaly_score_30_sig", "anomaly_score_60_sig", "anomaly_score_120_sig",]]
abnormal_transactions = abnormal_transactions.rename(columns={"anomaly_score_120_sig" : "anomaly_120",
                                                               "anomaly_score_60_sig" : "anomaly_60",
                                                               "anomaly_score_30_sig" : "anomaly_30"})
print("Loading data")

Loading data


___
## Feature extraction and engineering
___

In [45]:
abnormal_transactions.head()

,ACCESSION_NUMBER,TRANS_CODE,TRANS_ACQUIRED_DISP_CD,anomaly_30,anomaly_60,anomaly_120
0,0001127602-11-000075,S,D,-4.218320,-7.700075,-12.905019
1,0001127602-11-002229,S,D,-4.035856,-7.005213,-11.027958
2,0001127602-11-003431,S,D,-3.801423,-5.029422,-7.327263
3,0001127602-11-004376,S,D,-3.574648,-5.308864,-9.187093
4,0001127602-11-005631,S,D,-3.416012,-4.854250,-8.289322


In [156]:
abnormal_transactions["TRANS_ACQUIRED_DISP_CD"].count()

791687

In [227]:
df_features = abnormal_transactions.copy()

## Flip the signs if its sell
# df_features["anomaly_30"] = np.where(df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "D", -1 * df_features["anomaly_30"], df_features["anomaly_30"])
# df_features["anomaly_60"] = np.where(df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "D", -1 * df_features["anomaly_60"], df_features["anomaly_60"])
# df_features["anomaly_120"] = np.where(df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "D", -1 * df_features["anomaly_120"], df_features["anomaly_120"])

## Create a binary variable to extract Transcode = J
df_features["j_bin"] = np.where(df_features["TRANS_CODE"].str.upper() == "J", 1, 0)

## Create a binary variable to extract Transcode = J and Trans Acquired = D which means J code and sell
df_features["js_bin"] = np.where((df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "D") & (df_features["TRANS_CODE"].str.upper() == "J"), 1, 0)

## Create a binary variable to extract Transcode = J and Trans Acquired = A which means J code and buy
df_features["jb_bin"] = np.where((df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "A") & (df_features["TRANS_CODE"].str.upper() == "J"), 1, 0)

## Create a binary variable to extract Transcode = S which is sell
df_features["s_bin"] = np.where(df_features["TRANS_CODE"].str.upper() == "S", 1, 0)

## Create a binary variable to extract Transcode = S which is sell
df_features["b_bin"] = np.where(df_features["TRANS_CODE"].str.upper() == "P", 1, 0)

## Create a binary variable to extract Transcode Not J or S but Trans Acquired = D which is sell but non S or J coded
df_features["os_bin"] = np.where((~df_features["TRANS_CODE"].str.upper().isin(["S", "J"])) & (df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "D"), 1, 0)

## Create a binary variable to extract Transcode Not J or P but Trans Acquired = A which is buy but non P or J coded
df_features["ob_bin"] = np.where((~df_features["TRANS_CODE"].str.upper().isin(["P", "J"])) & (df_features["TRANS_ACQUIRED_DISP_CD"].str.upper() == "A"), 1, 0)

In [228]:
print(f'rows with binary j: {sum(df_features["j_bin"])}')
print(f'rows with binary j and sell: {sum(df_features["js_bin"])} \t rows with regular sell: {sum(df_features["s_bin"])} \t rows with non-regular sell: {sum(df_features["os_bin"])}')
print(f'rows with binary j and buy: {sum(df_features["jb_bin"])} \t rows with regular buy: {sum(df_features["b_bin"])} \t\t rows with non-regular buy: {sum(df_features["ob_bin"])}')


rows with binary j: 35785
rows with binary j and sell: 3082 	 rows with regular sell: 613936 	 rows with non-regular sell: 794
rows with binary j and buy: 32703 	 rows with regular buy: 139351 		 rows with non-regular buy: 1821


In [229]:
total_sell = sum(df_features["js_bin"]) + sum(df_features["s_bin"]) + sum(df_features["os_bin"])
total_buy = sum(df_features["jb_bin"]) + sum(df_features["b_bin"]) + sum(df_features["ob_bin"])
print(f"All cases accounted for: {len(df_features) == total_sell + total_buy}")

All cases accounted for: True


___
## Comparison
___

In [230]:
metrics = [
    "anomaly_30", "anomaly_60", "anomaly_120"
]

In [231]:
##################################
# Define functions
##################################

def compare_complement(df, col, metrics):
    split_1 = df[df[col] == 1]
    split_2 = df[df[col] != 1]
    
    for metric in metrics:
        A = split_1[metric].dropna()
        
        # print(A)
        B = split_2[metric].dropna()
        t_stat, p_val = stats.ttest_ind(A, B, equal_var=False)
        sign = "negative" if t_stat < 0 else "positive"
        print(f"{metric}: is more {sign} when {col} is separated\n" 
              f"t-statistic = {t_stat}, p-value = {p_val}\n"
              f"mean {col} = {np.mean(A)}, mean {col} complement = {np.mean(B)}\n")
        
    print(f"{col}: n = {len(A)}, {col} complement: n = {len(B)}")

def compare_columns(df, col1, col2, metrics):
    split_1 = df[df[col1] == 1]
    split_2 = df[df[col2] == 1]
    
    for metric in metrics:
        A = split_1[metric].dropna()
        
        # print(A)
        B = split_2[metric].dropna()
        t_stat, p_val = stats.ttest_ind(A, B, equal_var=False)
        sign = "negative" if t_stat < 0 else "positive"
        print(f"{metric}: is more {sign} when {col1} compared to {col2}\n" 
              f"t-statistic = {t_stat}, p-value = {p_val}\n"
              f"mean {col1} = {np.mean(A)}, mean {col2} = {np.mean(B)}\n")
    print(f"{col1}: n = {len(A)}, {col2} : n = {len(B)}")
    

In [232]:
## The inclusion of j code is significant to the anomaly level at 0.05 level
compare_complement(df_features, "j_bin", metrics)

anomaly_30: is more negative when j_bin is separated
t-statistic = -19.502858223630998, p-value = 2.6926491252973437e-84
mean j_bin = -4.940842239683503, mean j_bin complement = -2.010664534267951

anomaly_60: is more negative when j_bin is separated
t-statistic = -18.45380801171215, p-value = 1.0444830060206295e-75
mean j_bin = -9.363042599715834, mean j_bin complement = -3.911300535771447

anomaly_120: is more negative when j_bin is separated
t-statistic = -18.166109258970184, p-value = 1.961994725336045e-73
mean j_bin = -17.348050060771577, mean j_bin complement = -6.939656244339847

j_bin: n = 34501, j_bin complement: n = 730014


## Compare sells

In [233]:
##################################
# Compare Regular Sell to other-coded Sell
##################################

compare_columns(df_features, "s_bin", "os_bin", metrics)

anomaly_30: is more negative when s_bin compared to os_bin
t-statistic = -7.170741796007411, p-value = 1.7338687568896564e-12
mean s_bin = -0.2479462969715044, mean os_bin = 8.57114379802289

anomaly_60: is more negative when s_bin compared to os_bin
t-statistic = -7.188991582498718, p-value = 1.5282445841536905e-12
mean s_bin = -0.43991084129205793, mean os_bin = 16.92560442611971

anomaly_120: is more negative when s_bin compared to os_bin
t-statistic = -7.047914762203293, p-value = 3.9950846683005845e-12
mean s_bin = -0.15515792730261946, mean os_bin = 32.72410391794493

s_bin: n = 600046, os_bin : n = 781


In [234]:
##################################
# Compare Regular Sell to J-coded Sell
##################################

compare_columns(df_features, "s_bin", "js_bin", metrics)

anomaly_30: is more negative when s_bin compared to js_bin
t-statistic = -12.65636129493951, p-value = 9.146885826099802e-36
mean s_bin = -0.2479462969715044, mean js_bin = 6.07492006275562

anomaly_60: is more negative when s_bin compared to js_bin
t-statistic = -13.03806432991146, p-value = 8.589966121164844e-38
mean s_bin = -0.43991084129205793, mean js_bin = 12.328695939656926

anomaly_120: is more negative when s_bin compared to js_bin
t-statistic = -13.22384294907916, p-value = 8.430146805815766e-39
mean s_bin = -0.15515792730261946, mean js_bin = 25.136525575489028

s_bin: n = 600046, js_bin : n = 2859


### J-Coded sells does not have significance compared to Other-Coded sells

In [235]:
##################################
# Compare J-coded Sell to Others-sell 
##################################

compare_columns(df_features, "js_bin", "os_bin", metrics)

anomaly_30: is more negative when js_bin compared to os_bin
t-statistic = -1.8815691273408996, p-value = 0.06017139863219919
mean js_bin = 6.07492006275562, mean os_bin = 8.57114379802289

anomaly_60: is more negative when js_bin compared to os_bin
t-statistic = -1.7646452435802311, p-value = 0.07791444836459688
mean js_bin = 12.328695939656926, mean os_bin = 16.92560442611971

anomaly_120: is more negative when js_bin compared to os_bin
t-statistic = -1.5057921143951314, p-value = 0.13241985120185348
mean js_bin = 25.136525575489028, mean os_bin = 32.72410391794493

js_bin: n = 2859, os_bin : n = 781


## Compare buys

In [236]:
##################################
# Compare Regular Buy to J-coded Buy
##################################

compare_columns(df_features, "b_bin", "jb_bin", metrics)

anomaly_30: is more negative when b_bin compared to jb_bin
t-statistic = -23.587697284960093, p-value = 1.6972315412042312e-122
mean b_bin = -10.29148529763409, mean jb_bin = -5.9348794908734925

anomaly_60: is more negative when b_bin compared to jb_bin
t-statistic = -24.444013584450996, p-value = 2.321659280843773e-131
mean b_bin = -20.183002613500083, mean jb_bin = -11.32097163712962

anomaly_120: is more negative when b_bin compared to jb_bin
t-statistic = -24.90222094431647, p-value = 3.1327646849473786e-136
mean b_bin = -38.65480213394195, mean jb_bin = -21.186726558593122

b_bin: n = 127397, jb_bin : n = 31642


In [237]:
##################################
# Compare Regular Buy to other-coded Buy
##################################

compare_columns(df_features, "b_bin", "ob_bin", metrics)

anomaly_30: is more positive when b_bin compared to ob_bin
t-statistic = 0.9558593880556865, p-value = 0.33926596416154087
mean b_bin = -10.29148529763409, mean ob_bin = -10.916689038099607

anomaly_60: is more positive when b_bin compared to ob_bin
t-statistic = 1.6576403401173525, p-value = 0.09755651585333587
mean b_bin = -20.183002613500083, mean ob_bin = -22.285869912451833

anomaly_120: is more positive when b_bin compared to ob_bin
t-statistic = 1.1432595614445478, p-value = 0.25307530075859463
mean b_bin = -38.65480213394195, mean ob_bin = -41.33855732480276

b_bin: n = 127397, ob_bin : n = 1790


In [238]:
##################################
# Compare J-coded Buy to other-coded Buy
##################################

compare_columns(df_features, "j_bin", "ob_bin", metrics)

anomaly_30: is more positive when j_bin compared to ob_bin
t-statistic = 9.02572257479336, p-value = 4.122309592292695e-19
mean j_bin = -4.940842239683503, mean ob_bin = -10.916689038099607

anomaly_60: is more positive when j_bin compared to ob_bin
t-statistic = 10.05815012711405, p-value = 3.02074827555912e-23
mean j_bin = -9.363042599715834, mean ob_bin = -22.285869912451833

anomaly_120: is more positive when j_bin compared to ob_bin
t-statistic = 10.076731742426125, p-value = 2.490875140994702e-23
mean j_bin = -17.348050060771577, mean ob_bin = -41.33855732480276

j_bin: n = 34501, ob_bin : n = 1790


### Sell
Propose to split sell into 2 Features: Regular sell and non-regular sell (Includes J and the rest except S)

### Buy
Propose to split sell into 2 Features: J-coded buy and non J-coded buy (Includes P and the rest except J)

Possibly another variable for the significance 

___
## Checking significance in model
___

In [239]:
df_features.head()

,ACCESSION_NUMBER,TRANS_CODE,TRANS_ACQUIRED_DISP_CD,anomaly_30,anomaly_60,anomaly_120,j_bin,js_bin,jb_bin,s_bin,b_bin,os_bin,ob_bin
0,0001127602-11-000075,S,D,-4.218320,-7.700075,-12.905019,0,0,0,1,0,0,0
1,0001127602-11-002229,S,D,-4.035856,-7.005213,-11.027958,0,0,0,1,0,0,0
2,0001127602-11-003431,S,D,-3.801423,-5.029422,-7.327263,0,0,0,1,0,0,0
3,0001127602-11-004376,S,D,-3.574648,-5.308864,-9.187093,0,0,0,1,0,0,0
4,0001127602-11-005631,S,D,-3.416012,-4.854250,-8.289322,0,0,0,1,0,0,0


In [ ]:
df_features["regular_sell_bin"] = df_features["s_bin"]
df_features["non_regular_sell_bin"] = df_features["js_bin"] | df_features["os_bin"]

df_features["j_buy"] = df_features["jb_bin"]
df_features["non_j_buy_bin"] = df_features["b_bin"] | df_features["ob_bin"]

columns_to_keep = [
    "regular_sell_bin", "non_regular_sell_bin", "j_buy", "non_j_buy_bin",
    "anomaly_30", "anomaly_60", "anomaly_120"
]

df_new_features = df_features.copy()
df_new_features = df_new_features[columns_to_keep]

print(f"Check correctness\n"
      f'Sell: {sum(df_new_features["regular_sell_bin"]) + sum(df_new_features["non_regular_sell_bin"]) == sum(df_features["TRANS_ACQUIRED_DISP_CD"]=="D")}\n'
      f'Buy: {sum(df_new_features["j_buy"]) + sum(df_new_features["non_j_buy_bin"]) == sum(df_features["TRANS_ACQUIRED_DISP_CD"]=="A")}')

Check correctness
Sell: True
Buy: True


In [241]:
def find_significance(data, columns, label):
    data = data.dropna(subset=[label])
    X = data[columns]
    y = data[label]
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=123)
    X_with_const = sm.add_constant(train_x)
    model = sm.OLS(train_y, X_with_const)
    results = model.fit()

    print(results.summary())

In [242]:
variables = ["regular_sell_bin", "non_regular_sell_bin", "j_buy", "non_j_buy_bin"]
label = ["anomaly_30", "anomaly_60", "anomaly_120"]

In [243]:
find_significance(df_new_features, variables, label[1])

                            OLS Regression Results                            
Dep. Variable:             anomaly_60   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     3070.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        12:42:37   Log-Likelihood:            -3.2818e+06
No. Observations:              607382   AIC:                         6.564e+06
Df Residuals:                  607377   BIC:                         6.564e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -5.425e+11 

In [244]:
import statsmodels.api as sm
import numpy as np

# Example Data (same as before)
X =  train_x # Feature (independent variable)
y =  train_y   # Target (dependent variable)

# Add a constant (intercept) term to the feature matrix
X_with_const = sm.add_constant(X)

# Fit the OLS (Ordinary Least Squares) model
model = sm.OLS(y, X_with_const)
results = model.fit()

# Print the summary of the model
print(results.summary())

NameError: name 'train_x' is not defined

### Propose a further of splitting of J to J-buy and J-sell, Reg-Buy and Reg-Sell, Other-Buy and Other-sell

In [ ]:
# df_footnote = footnotes.copy()
# df_footnote = df_footnote.sort_values(by=["ACCESSION_NUMBER", "FOOTNOTE_ID"])
# df_footnote["FOOTNOTE_TXT"] = df_footnote["FOOTNOTE_TXT"].astype(str)
# df_grouped = df_footnote.groupby("ACCESSION_NUMBER", sort=True)["FOOTNOTE_TXT"].apply(lambda x: " ".join(x)).reset_index()
# df_grouped

In [ ]:
# df_footnote_combined = pd.merge(abnormal_transactions, df_grouped, how = "left", left_on = "ACCESSION_NUMBER", right_on = "ACCESSION_NUMBER")
# df_footnote_combined.head()

In [ ]:
# df_footnote_combined = df_footnote_combined[df_footnote_combined["FOOTNOTE_TXT"].notnull()]
# df_footnote_combined.head()

## Create labels (Can be modified later)

In [ ]:
# df_footnote_combined["signed_anomaly_score"] = \
#     np.where(df_footnote_combined["TRANS_ACQUIRED_DISP_CD"] == "D", -1 * df_footnote_combined["anomaly_score"],  df_footnote_combined["anomaly_score"])

# threshold = df_footnote_combined["signed_anomaly_score"].quantile(0.95)

# df_footnote_combined["label"] = np.where(df_footnote_combined["signed_anomaly_score"] > threshold, 1, 0)
# df_footnote_combined

In [ ]:
# df_footnote_exist_drop = df_footnote_combined.dropna(subset=["anomaly_score"])
# df_footnote_exist_drop.shape

___
## J-Code comparison
___

In [ ]:
df_j_code = df_footnote_exist_drop[df_footnote_exist_drop["TRANS_CODE"] == "J"]
df_non_j_code =  df_footnote_exist_drop[df_footnote_exist_drop["TRANS_CODE"] != "J"]
print(f"Shape J code: {df_j_code.shape}, \t Shape non-J code: {df_non_j_code.shape}")

Shape J code: (33793, 7), 	 Shape non-J code: (580121, 7)


In [ ]:
df_footnote_exist_drop_copy = df_footnote_exist_drop.copy()
df_footnote_exist_drop_copy['J_Code'] = np.where(df_footnote_exist_drop_copy["TRANS_CODE"] == "J", "J", "Others")

In [ ]:
df_footnote_exist_drop_copy

,ACCESSION_NUMBER,anomaly_score,TRANS_CODE,TRANS_ACQUIRED_DISP_CD,FOOTNOTE_TXT,signed_anomaly_score,label,J_Code
0,0001127602-11-000075,-12.905019,S,D,The weighted average sales price was $41.652 w...,12.905019,0,Others
1,0001127602-11-002229,-11.027958,S,D,This transaction was effected pursuant to a Ru...,11.027958,0,Others
2,0001127602-11-003431,-7.327263,S,D,This transaction was effected pursuant to a Ru...,7.327263,0,Others
3,0001127602-11-004376,-9.187093,S,D,This transaction was effected pursuant to a Ru...,9.187093,0,Others
4,0001127602-11-005631,-8.289322,S,D,This transaction was effected pursuant to a Ru...,8.289322,0,Others
...,...,...,...,...,...,...,...,...
791678,0001104659-20-056286,-4.941855,S,D,Represents a restricted stock award issued pur...,4.941855,0,Others
791679,0001104659-20-056286,-4.941855,S,D,Represents a restricted stock award issued pur...,4.941855,0,Others
791680,0001104659-20-056286,-4.941855,S,D,Represents a restricted stock award issued pur...,4.941855,0,Others
791681,0001104659-20-056286,-5.040325,S,D,Represents a restricted stock award issued pur...,5.040325,0,Others


In [ ]:
count = df_footnote_exist_drop_copy[df_footnote_exist_drop_copy["J_Code"] == "J"].shape[0]
pct_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] == "J") & (df_footnote_exist_drop_copy["label"] == 1)].shape[0]  /  count

In [ ]:
count = df_footnote_exist_drop_copy[df_footnote_exist_drop_copy["J_Code"] != "J"].shape[0]
pct_non_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] != "J") & (df_footnote_exist_drop_copy["label"] == 1)].shape[0]  /  count

In [ ]:
print(f"Pct of anomalies with J codes: {pct_J}\nPct of anomalies with non J codes: {pct_non_J}")

Pct of anomalies with J codes: 0.028260290592726306
Pct of anomalies with non J codes: 0.0512668908727662


In [ ]:
count = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] == "J") & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "D")].shape[0]
pct_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] == "J") & (df_footnote_exist_drop_copy["label"] == 1) & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "D")].shape[0]  /  count

In [ ]:
count = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] != "J") & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "D")].shape[0]
pct_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] != "J") & (df_footnote_exist_drop_copy["label"] == 1) & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "D")].shape[0]  /  count

In [ ]:
print(f"Pct of anomalies with J codes: {pct_J}\nPct of anomalies with non J codes: {pct_non_J}")

Pct of anomalies with J codes: 0.053414366772091486
Pct of anomalies with non J codes: 0.0512668908727662


In [ ]:
count = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] == "J") & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "A")].shape[0]
pct_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] == "J") & (df_footnote_exist_drop_copy["label"] == 1) & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "A")].shape[0]  /  count

In [ ]:
count = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] != "J") & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "A")].shape[0]
pct_J = df_footnote_exist_drop_copy[(df_footnote_exist_drop_copy["J_Code"] != "J") & (df_footnote_exist_drop_copy["label"] == 1) & (df_footnote_exist_drop_copy["TRANS_ACQUIRED_DISP_CD"] == "A")].shape[0]  /  count

In [ ]:
print(f"Pct of anomalies with J codes: {pct_J}\nPct of anomalies with non J codes: {pct_non_J}")

Pct of anomalies with J codes: 0.036115971007248185
Pct of anomalies with non J codes: 0.0512668908727662
